In [75]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, Callback

import torch
from torch import nn

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, RecurrentNetwork
from pytorch_forecasting.metrics.quantile import QuantileLoss
import pytorch_forecasting.metrics

import pandas as pd

In [99]:
csv_file = open("2016_2022.csv", "a")
csv_file.write("\n")
csv_file.write("")
csv_file.close()

In [100]:
# Load Data
datas = pd.read_csv("2016_2022.csv")
data = datas[datas["Source"] == 3]
n = len(data)
time_idx = [i for i in range(n)]
data.insert(3, "Time", time_idx, True)
data = data.drop("Date", axis = 1)
print(data)

         Valeur  Source    Time
1         3.607       3       0
13        3.496       3       1
25        3.446       3       2
37        3.298       3       3
49        3.229       3       4
...         ...     ...     ...
4034625   1.931       3  338248
4034637   1.915       3  338249
4034649   1.918       3  338250
4034661   1.942       3  338251
4034673   1.998       3  338252

[338253 rows x 3 columns]


In [101]:
# define dataset
max_encoder_length = 10000
max_prediction_length = 2

In [102]:
training_cutoff = data["Time"].max() - max_prediction_length
print(training_cutoff)
#print(data[lambda x: x.Time <= training_cutoff])
training = TimeSeriesDataSet(
    data[lambda x: x.Time <= training_cutoff],
    time_idx= "Time",
    target= "Valeur",
    group_ids=["Source"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_reals=["Source"],
    time_varying_known_reals=["Time"],
    time_varying_unknown_reals=["Valeur"],
)
print(training)

338250
TimeSeriesDataSet[length=328250](
	time_idx='Time',
	target='Valeur',
	group_ids=['Source'],
	weight=None,
	max_encoder_length=10000,
	min_encoder_length=10000,
	min_prediction_idx=0,
	min_prediction_length=2,
	max_prediction_length=2,
	static_categoricals=[],
	static_reals=['Source'],
	time_varying_known_categoricals=[],
	time_varying_known_reals=['Time'],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['Valeur'],
	variable_groups={},
	constant_fill_strategy={},
	allow_missing_timesteps=False,
	lags={},
	add_relative_time_idx=False,
	add_target_scales=False,
	add_encoder_length=False,
	target_normalizer=EncoderNormalizer(
	method='standard',
	center=True,
	max_length=None,
	transformation='relu',
	method_kwargs={}
),
	categorical_encoders={'__group_id__Source': NaNLabelEncoder(add_nan=False, warn=True)},
	scalers={'Source': StandardScaler(), 'Time': StandardScaler()},
	randomize_length=None,
	predict_mode=False
)


In [103]:
# create validation and training dataset
validation = TimeSeriesDataSet.from_dataset(training, data, min_prediction_idx=training.index.time.max() + 1, stop_randomization=True)
batch_size = 32
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=8)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=8)
print(validation)

TimeSeriesDataSet[length=10002](
	time_idx='Time',
	target='Valeur',
	group_ids=['Source'],
	weight=None,
	max_encoder_length=10000,
	min_encoder_length=10000,
	min_prediction_idx=328250,
	min_prediction_length=2,
	max_prediction_length=2,
	static_categoricals=[],
	static_reals=['Source'],
	time_varying_known_categoricals=[],
	time_varying_known_reals=['Time'],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['Valeur'],
	variable_groups={},
	constant_fill_strategy={},
	allow_missing_timesteps=False,
	lags={},
	add_relative_time_idx=False,
	add_target_scales=False,
	add_encoder_length=False,
	target_normalizer=EncoderNormalizer(
	method='standard',
	center=True,
	max_length=None,
	transformation='relu',
	method_kwargs={}
),
	categorical_encoders={'__group_id__Source': NaNLabelEncoder(add_nan=False, warn=True)},
	scalers={'Source': StandardScaler(), 'Time': StandardScaler()},
	randomize_length=None,
	predict_mode=False
)


In [104]:
class PrintCallback(Callback):
    def on_train_start(self, trainer, pl_module):
        print("Training is started!")
    def on_train_end(self, trainer, pl_module):
        print("Training is done.")

In [105]:
# define trainer with early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", patience=1, verbose=1, mode="min")
lr_logger = LearningRateMonitor()
trainer = pl.Trainer(
    max_epochs=100,
    gradient_clip_val=0.1,
    limit_train_batches=10,
    callbacks=[lr_logger, early_stop_callback, PrintCallback()],
    accelerator = 'gpu',
    devices = 1,
    auto_lr_find = False,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [106]:
# create the model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=8,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=4,
    output_size=7,
    loss=QuantileLoss(),
    log_interval=1,
    reduce_on_plateau_patience=4
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

/home/alexandre/miniconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/home/alexandre/miniconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


Number of parameters in network: 4.6k


In [98]:
# find optimal learning rate (set limit_train_batches to 1.0 and log_interval = -1)
res = trainer.tuner.lr_find(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, early_stop_threshold=1000.0, max_lr=0.3,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

/home/alexandre/miniconda3/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:601: UserWarning: Checkpoint directory /mnt/c/Users/alex/Documents/COURS/PIIA-Marees/PyTorch/lightning_logs/version_24/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_steps=100` reached.
LR finder stopped early after 350 steps due to diverging loss.
Restoring states from the checkpoint path at /mnt/c/Users/alex/Documents/COURS/PIIA-Marees/PyTorch/.lr_find_0f6a6b45-4708-4ce0-8650-0403750dce91.ckpt


KeyError: 'radam_buffer'

In [ ]:
# fit the model
trainer.fit(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 24    
4  | static_variable_selection          | VariableSelectionNetwork        | 168   
5  | encoder_variable_selection         | VariableSelectionNetwork        | 398   
6  | decoder_variable_selection         | VariableSelectionNetwork        | 168   
7  | static_context_variable_selection  | GatedResidualNetwork            | 304   
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 304   
9  | static_context_initial_cell_lstm 

/home/alexandre/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1558: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training is started!
Epoch 0:   1%|▏                           | 2/323 [00:33<1:30:33, 16.93s/it, loss=1.54, v_num=25, train_loss_step=1.480]